## Dataset: https://www.kaggle.com/datasets/saurabhbadole/latest-data-science-job-salaries-2024/

In [14]:
import pandas as pd

p = pd.read_csv('../data/jobs_in_data_2020_2024.csv')

## Data Exploration

In [15]:
p

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2021,MI,FT,Data Scientist,30400000,CLP,40038,CL,100,CL,L
1,2021,MI,FT,BI Data Analyst,11000000,HUF,36259,HU,50,US,L
2,2020,MI,FT,Data Scientist,11000000,HUF,35735,HU,50,HU,L
3,2021,MI,FT,ML Engineer,8500000,JPY,77364,JP,50,JP,S
4,2022,SE,FT,Lead Machine Learning Engineer,7500000,INR,95386,IN,50,IN,L
...,...,...,...,...,...,...,...,...,...,...,...
14833,2022,MI,FT,Business Intelligence Developer,15000,USD,15000,GH,100,GH,M
14834,2020,EX,FT,Staff Data Analyst,15000,USD,15000,NG,0,CA,M
14835,2021,EN,FT,Machine Learning Developer,15000,USD,15000,TH,100,TH,L
14836,2022,EN,FT,Data Analyst,15000,USD,15000,ID,0,ID,L


In [16]:
p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14838 entries, 0 to 14837
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           14838 non-null  int64 
 1   experience_level    14838 non-null  object
 2   employment_type     14838 non-null  object
 3   job_title           14838 non-null  object
 4   salary              14838 non-null  int64 
 5   salary_currency     14838 non-null  object
 6   salary_in_usd       14838 non-null  int64 
 7   employee_residence  14838 non-null  object
 8   remote_ratio        14838 non-null  int64 
 9   company_location    14838 non-null  object
 10  company_size        14838 non-null  object
dtypes: int64(4), object(7)
memory usage: 1.2+ MB


In [17]:
p.nunique()

work_year                5
experience_level         4
employment_type          4
job_title              153
salary                2363
salary_currency         23
salary_in_usd         2730
employee_residence      88
remote_ratio             3
company_location        77
company_size             3
dtype: int64

In [18]:
p.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [19]:
p.describe()

,work_year,salary,salary_in_usd,remote_ratio
count,14838.000000,1.483800e+04,14838.000000,14838.000000
mean,2023.138900,1.650227e+05,149874.718763,32.760480
std,0.700799,3.562354e+05,69009.181349,46.488278
min,2020.000000,1.400000e+04,15000.000000,0.000000
25%,2023.000000,1.021000e+05,102000.000000,0.000000
50%,2023.000000,1.422000e+05,141300.000000,0.000000
75%,2024.000000,1.875000e+05,185900.000000,100.000000
max,2024.000000,3.040000e+07,800000.000000,100.000000


In [20]:
# - https://www.kaggle.com/code/murilozangari/jobs-data-field-2024-eda-salary-estimation

# Check for duplicates
duplicate_rows = p[p.duplicated()]

# Print the number of duplicates
num_duplicates = duplicate_rows.shape[0]
print(f"Number of duplicate rows:", num_duplicates, "\nPercentege from the total:", round(num_duplicates/len(p),3)*100)

Number of duplicate rows: 5711 
Percentege from the total: 38.5


In [21]:
rows_before = len(p)
p = p.drop_duplicates()
rows_after = len(p)

print(f"Number of rows deleted: {rows_before - rows_after}")

Number of rows deleted: 5711


# Preprocessing

In [22]:
p['work_year'] = p['work_year'].astype(str)
p.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9127 entries, 0 to 14837
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           9127 non-null   object
 1   experience_level    9127 non-null   object
 2   employment_type     9127 non-null   object
 3   job_title           9127 non-null   object
 4   salary              9127 non-null   int64 
 5   salary_currency     9127 non-null   object
 6   salary_in_usd       9127 non-null   int64 
 7   employee_residence  9127 non-null   object
 8   remote_ratio        9127 non-null   int64 
 9   company_location    9127 non-null   object
 10  company_size        9127 non-null   object
dtypes: int64(3), object(8)
memory usage: 855.7+ KB


C:\Users\Slakes\AppData\Local\Temp\ipykernel_2976\2623057264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p['work_year'] = p['work_year'].astype(str)


In [23]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Separate the target variable from the predictors
X = p.drop('salary', axis=1)  #
X = p.drop('salary_in_usd', axis=1)  
# Set target variable
y = p['salary_in_usd'] 

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

# Identify the categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Identify the numerical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', OneHotEncoder(handle_unknown='ignore'))
])

# Create preprocessor
preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_cols), ('passthrough', 'passthrough', numerical_cols)])

# Create the XGBoost regressor
model = XGBRegressor()

# Create the pipeline
regressor = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# XGBoost using grid search

In [24]:
import time

# as proposed by https://doi.org/10.1007/s00521-019-04566-2 and https://doi.org/10.1007/s10064-022-02708-w
param_grid = {
    'model__min_child_weight': [1, 5, 10],
    'model__gamma': [0, 0.2, 0.5, 1, 1.5, 2, 5],
    'model__subsample': [0.6, 0.8, 1.0],
    'model__colsample_bytree': [0.6, 0.8, 1.0],
    'model__max_depth': [3, 4, 5, 6],
    'model__learning_rate': [0.01, 0.05, 0.1], 
    'model__n_estimators': [100, 200, 300, 400, 500]
}

grid_search = GridSearchCV(regressor, param_grid, cv=5, n_jobs=-1, verbose=2)

start_time = time.time()
grid_search.fit(X_train, y_train)
end_time = time.time()

Fitting 5 folds for each of 11340 candidates, totalling 56700 fits


In [25]:
# Print optimal configuration
print("Optimal Configuration:", grid_search.best_params_)

# Fit the model with the optimal configuration
cv_results = cross_validate(grid_search.best_estimator_, X_train, y_train, cv=5, scoring='r2')
print(abs(cv_results['test_score']))

Optimal Configuration: {'model__colsample_bytree': 1.0, 'model__gamma': 0, 'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__min_child_weight': 1, 'model__n_estimators': 400, 'model__subsample': 0.8}
[0.9887974  0.98002262 0.99327476 0.98707238 0.97698795]


In [26]:
test_predictions = grid_search.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
test_r2 =r2_score(y_test, test_predictions)

print("Test RMSE Score:", test_rmse)
print("Test NRMSE score:", test_rmse / (y.max() - y.min()))
print("Test R2 Score:", test_r2)
print(f"Tuning completed in: {end_time-start_time:.6f} seconds.")

Test RMSE Score: 13073.592545943575
Test NRMSE score: 0.016654258020310286
Test R2 Score: 0.9688547255909276
Tuning completed in: 2005.018313 seconds.


In [27]:
avg_cv_result = np.mean(np.abs(cv_results['test_score']))
print(avg_cv_result)

0.985231023721312
